In [6]:
import pandas as pd
import json

In [10]:
df = pd.read_json("train.jsonl", lines=True)
df

,id,abstract,summary,generated
0,train_0001,This paper addresses the problem of evaluating...,We show that rare but catastrophic failures ma...,This paper addresses the problem of evaluating...
1,train_0002,We explore efficient neural architecture searc...,In this paper we propose a hierarchical archit...,We explore efficient neural architecture searc...
2,train_0003,We propose Episodic Backward Update - a new al...,"We propose Episodic Backward Update, a novel d...",We propose Episodic Backward Update - a new al...
3,train_0004,Animals develop novel skills not only through ...,A new RL algorithm called Interior Policy Diff...,Animals develop novel skills not only through ...
4,train_0005,Deep generative models such as Generative Adve...,We investigate the modularity of deep generati...,Deep generative models such as Generative Adve...
...,...,...,...,...
2344,train_2345,"In this paper, we present a layer-wise learnin...",Learning a better neural networks' representat...,"In this paper, we present a layer-wise learnin..."
2345,train_2346,Brain-Computer Interfaces (BCI) may help patie...,We present an open-loop brain-machine interfac...,Brain-Computer Interfaces (BCI) may help patie...
2346,train_2347,We outline the problem of concept drifts for t...,We introduce an augmented robust feature space...,We outline the problem of concept drifts for t...
2347,train_2348,Cold-start and efficiency issues of the Top-k ...,It can generate effective hash codes for effic...,Cold-start and efficiency issues of the Top-k ...


In [13]:
# Create two separate DataFrames
human_df = df[['id', 'abstract', 'summary']].copy()
human_df['label'] = 0
human_df.rename(columns={'summary': 'text'}, inplace=True)

ai_df = df[['id', 'abstract', 'generated']].copy()
ai_df['label'] = 1
ai_df.rename(columns={'generated': 'text'}, inplace=True)

# Concatenate the two DataFrames
combined_df = pd.concat([human_df, ai_df], ignore_index=True)
combined_df

,id,abstract,text,label
0,train_0001,This paper addresses the problem of evaluating...,We show that rare but catastrophic failures ma...,0
1,train_0002,We explore efficient neural architecture searc...,In this paper we propose a hierarchical archit...,0
2,train_0003,We propose Episodic Backward Update - a new al...,"We propose Episodic Backward Update, a novel d...",0
3,train_0004,Animals develop novel skills not only through ...,A new RL algorithm called Interior Policy Diff...,0
4,train_0005,Deep generative models such as Generative Adve...,We investigate the modularity of deep generati...,0
...,...,...,...,...
4693,train_2345,"In this paper, we present a layer-wise learnin...","In this paper, we present a layer-wise learnin...",1
4694,train_2346,Brain-Computer Interfaces (BCI) may help patie...,Brain-Computer Interfaces (BCI) may help patie...,1
4695,train_2347,We outline the problem of concept drifts for t...,We outline the problem of concept drifts for t...,1
4696,train_2348,Cold-start and efficiency issues of the Top-k ...,Cold-start and efficiency issues of the Top-k ...,1


In [14]:
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [17]:
# Split the dataset into train and evaluation sets
train_df, eval_df = train_test_split(combined_df[['text', 'label']], test_size=0.2, random_state=42)

model = 'distilbert'
model_name = 'distilbert-base-uncased'
# Set up the model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 3
model_args.learning_rate = 2e-5
model_args.overwrite_output_dir = True
model_args.train_batch_size = 16
model_args.eval_batch_size = 16
model_args.evaluate_during_training = True

# Train the classification model using a pre-trained model (e.g., 'distilbert-base-uncased')
model = ClassificationModel(model, model_name, num_labels=2, args=model_args)
model.train_model(train_df, eval_df=eval_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
print(result)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

  0%|          | 0/3758 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/235 [00:00<?, ?it/s]

c:\Users\hp\miniconda3\envs\Pytorch\lib\site-packages\simpletransformers\classification\classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


...

  0%|          | 0/940 [00:00<?, ?it/s]

c:\Users\hp\miniconda3\envs\Pytorch\lib\site-packages\simpletransformers\classification\classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/940 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/59 [00:00<?, ?it/s]

{'mcc': 1.0, 'tp': 472, 'tn': 468, 'fp': 0, 'fn': 0, 'auroc': 1.0, 'auprc': 0.9999999999999999, 'eval_loss': 0.0003645298425384434}


In [18]:
predictions1, raw_outputs1 = model.predict(["We show that rare but catastrophic failures ma..."])
predictions

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([0], dtype=int64)

In [41]:
test = pd.read_json("test.jsonl", lines=True)
test

,id,abstract,summary1,summary2
0,test_0001,This paper addresses the problem of evaluating...,The field of few-shot learning has recently se...,We show that rare but catastrophic failures ma...
1,test_0002,We explore efficient neural architecture searc...,Convolutional neural networks (CNNs) have been...,In this paper we propose a hierarchical archit...
2,test_0003,We propose Episodic Backward Update - a new al...,"We propose Episodic Backward Update, a novel d...",One of the distinguishing aspects of human lan...
3,test_0004,Animals develop novel skills not only through ...,A new RL algorithm called Interior Policy Diff...,Distributed computing can significantly reduce...
4,test_0005,Deep generative models such as Generative Adve...,We investigate the modularity of deep generati...,We investigate methods for semi-supervised lea...
...,...,...,...,...
256,test_0257,There have been multiple attempts with variati...,Words in natural language follow a Zipfian dis...,We present a generative model that proves stat...
257,test_0258,Effective performance of neural networks depen...,We introduce amortized proximal optimization (...,"In reinforcement learning, we can learn a mode..."
258,test_0259,We propose procedures for evaluating and stren...,The use of deep learning models as priors for ...,We propose procedures for evaluating and stren...
259,test_0260,We propose a new model for making generalizabl...,"In this paper, we propose a new control framew...",We propose a new model for making generalizabl...


In [20]:
summary1_df = test[['id', 'abstract', 'summary1']].copy()
summary1_df.rename(columns={'summary1': 'text'}, inplace=True)

summary2_df = test[['id', 'abstract', 'summary2']].copy()
summary2_df.rename(columns={'summary2': 'text'}, inplace=True)

In [21]:
summary1_df

,id,abstract,text
0,test_0001,This paper addresses the problem of evaluating...,The field of few-shot learning has recently se...
1,test_0002,We explore efficient neural architecture searc...,Convolutional neural networks (CNNs) have been...
2,test_0003,We propose Episodic Backward Update - a new al...,"We propose Episodic Backward Update, a novel d..."
3,test_0004,Animals develop novel skills not only through ...,A new RL algorithm called Interior Policy Diff...
4,test_0005,Deep generative models such as Generative Adve...,We investigate the modularity of deep generati...
...,...,...,...
256,test_0257,There have been multiple attempts with variati...,Words in natural language follow a Zipfian dis...
257,test_0258,Effective performance of neural networks depen...,We introduce amortized proximal optimization (...
258,test_0259,We propose procedures for evaluating and stren...,The use of deep learning models as priors for ...
259,test_0260,We propose a new model for making generalizabl...,"In this paper, we propose a new control framew..."


In [22]:
summary2_df

,id,abstract,text
0,test_0001,This paper addresses the problem of evaluating...,We show that rare but catastrophic failures ma...
1,test_0002,We explore efficient neural architecture searc...,In this paper we propose a hierarchical archit...
2,test_0003,We propose Episodic Backward Update - a new al...,One of the distinguishing aspects of human lan...
3,test_0004,Animals develop novel skills not only through ...,Distributed computing can significantly reduce...
4,test_0005,Deep generative models such as Generative Adve...,We investigate methods for semi-supervised lea...
...,...,...,...
256,test_0257,There have been multiple attempts with variati...,We present a generative model that proves stat...
257,test_0258,Effective performance of neural networks depen...,"In reinforcement learning, we can learn a mode..."
258,test_0259,We propose procedures for evaluating and stren...,We propose procedures for evaluating and stren...
259,test_0260,We propose a new model for making generalizabl...,We propose a new model for making generalizabl...


In [33]:
# Convert the 'text' column to a list of strings
summary1_texts = summary1_df['text'].astype(str).tolist()

# Make predictions on summary1_texts
summary1_preds, _ = model.predict(summary1_texts)

# Create a DataFrame containing the output
output_df_1 = pd.DataFrame({
    'id': summary1_df['id'],
    'text': summary1_df['text'],
    'predicted_label': summary1_preds
})

output_df_1


  0%|          | 0/261 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

,id,text,predicted_label
0,test_0001,The field of few-shot learning has recently se...,1
1,test_0002,Convolutional neural networks (CNNs) have been...,1
2,test_0003,"We propose Episodic Backward Update, a novel d...",0
3,test_0004,A new RL algorithm called Interior Policy Diff...,0
4,test_0005,We investigate the modularity of deep generati...,0
...,...,...,...
256,test_0257,Words in natural language follow a Zipfian dis...,1
257,test_0258,We introduce amortized proximal optimization (...,0
258,test_0259,The use of deep learning models as priors for ...,1
259,test_0260,"In this paper, we propose a new control framew...",1


In [32]:
# Convert the 'text' column to a list of strings
summary2_texts = summary2_df['text'].astype(str).tolist()

# Make predictions on summary1_texts
summary2_preds, _ = model.predict(summary2_texts)

# Create a DataFrame containing the output
output_df_2 = pd.DataFrame({
    'id': summary2_df['id'],
    'text': summary2_df['text'],
    'predicted_label': summary2_preds
})

output_df_2

  0%|          | 0/261 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

,id,text,predicted_label
0,test_0001,We show that rare but catastrophic failures ma...,0
1,test_0002,In this paper we propose a hierarchical archit...,0
2,test_0003,One of the distinguishing aspects of human lan...,1
3,test_0004,Distributed computing can significantly reduce...,1
4,test_0005,We investigate methods for semi-supervised lea...,1
...,...,...,...
256,test_0257,We present a generative model that proves stat...,0
257,test_0258,"In reinforcement learning, we can learn a mode...",1
258,test_0259,We propose procedures for evaluating and stren...,0
259,test_0260,We propose a new model for making generalizabl...,0


In [39]:
combined_preds = list(zip(summary1_preds, summary2_preds))
combined_preds

[(1, 0),
 (1, 0),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (1, 0),
 (1, 0),
 (1, 0),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (1, 0),
 (1, 0),
 (1, 0),
 (0, 1),
 (1, 0),
 (1, 0),
 (1, 0),
 (1, 0),
 (1, 0),
 (1, 0),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (1, 0),
 (0, 1),
 (1, 0),
 (1, 0),
 (0, 1),
 (1, 0),
 (1, 0),
 (1, 0),
 (1, 0),
 (1, 0),
 (1, 0),
 (0, 1),
 (1, 0),
 (0, 1),
 (1, 0),
 (0, 1),
 (1, 0),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (1, 0),
 (1, 0),
 (0, 1),
 (0, 1),
 (1, 0),
 (0, 1),
 (1, 0),
 (1, 0),
 (1, 0),
 (1, 0),
 (0, 1),
 (0, 1),
 (1, 0),
 (1, 0),
 (1, 0),
 (0, 1),
 (0, 1),
 (1, 0),
 (1, 0),
 (1, 0),
 (0, 1),
 (1, 0),
 (0, 1),
 (0, 1),
 (0, 1),
 (1, 0),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (1, 0),
 (1, 0),
 (1, 0),
 (1, 0),
 (0, 1),
 (1, 0),
 (1, 0),
 (1, 0),
 (0, 1),
 (0, 1),
 (0, 1),
 (1, 0),
 (1, 0),
 (1, 0),
 (0, 1),
 (1, 0),
 (1, 0),
 (1, 0),
 (0, 1),
 (1, 0),
 (0, 1),
 (0, 1),
 (1, 0),
 (0, 1),
 (1, 0),
 (1, 0),
 (0, 1),
 (1, 0),
 

In [40]:
binary_labels = [0 if pred == (0, 1) else 1 for pred in combined_preds]


In [44]:
binary_labels_df = pd.DataFrame(binary_labels, columns=['label'])

binary_labels_df

,label
0,1
1,1
2,0
3,0
4,0
...,...
256,1
257,0
258,1
259,1


In [45]:
submit = pd.read_csv("Sample_Submission.csv")
submit

,id,answer
0,test_0001,NaN
1,test_0002,NaN
2,test_0003,NaN
3,test_0004,NaN
4,test_0005,NaN
...,...,...
256,test_0257,NaN
257,test_0258,NaN
258,test_0259,NaN
259,test_0260,NaN


In [46]:
submit["answer"] = binary_labels_df["label"]
submit

,id,answer
0,test_0001,1
1,test_0002,1
2,test_0003,0
3,test_0004,0
4,test_0005,0
...,...,...
256,test_0257,1
257,test_0258,0
258,test_0259,1
259,test_0260,1


In [48]:
submit.to_csv(f"submit_distilbert-base-uncased.csv")